# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
#pip install citipy
import os
import csv
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import random
import seaborn as sns
import requests
import urllib
import time
import json
from pprint import pprint
from datetime import datetime
import scipy.stats as st
from scipy.stats import linregress
#pip install <citipy>

# Import API key
from API_keys import Weathermap_API_key

# Incorporated citipy to determine city based on latitude and longitude
from citipy import citipy

# Output File (CSV)
output_data_file = "output_data/cities.csv"

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

## Generate Cities List

In [2]:
# List for holding lat_lngs and cities
lat_lngs = []
cities = [] 

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
city_count = len(cities)
print(city_count)

629


In [ ]:
cities_df=pd.DataFrame(cities)
cities_df.columns=["City"]
cities_df['Cloudiness'] = ""
cities_df['Country'] = ""
cities_df['Date'] = ""
cities_df['Humidity'] = ""
cities_df['Lat']=""
cities_df['Lng']=""
cities_df['Max Temp']=""
cities_df['Wind Speed']=""
cities_df.head()

## Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [8]:

url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
# Build partial query URL
query_url = f"{url}appid={Weathermap_API_key}&units={units}&q="
print(query_url+"London")
#response = requests.get(query_url+"London").json()
#print(response)
new_cities = []
cloudiness = []
country = []
date = []
humidity = []
temp = []
lat = []
lng = []
wind = []
city_data = []
print("Beginning Data Retrieval ")
record_counter = 0
set_counter = 1
for city in cities:
    response = requests.get(query_url + city).json()
    if record_counter < 50:
        record_counter += 1
    else:
        set_counter +=1
        record_counter = 0
    print('Processing record {} of set {} | {}'.format(record_counter, set_counter, city))
    print(url)
    try:
        new_cities.append(response['name'])
        cloudiness.append(response['clouds'] ['all'])
        country.append(response['sys'] ['country'])
        date.append(response['date'])
        humidity.append(response['main'] ['humidity'])
        lat.append(response['coord'] ['lat'])
        lng.append(response['coord'] ['lon'])
        temp.append(response['main'] ['temp_max'])
        wind.append(response['wind'] ['speed'])
        time.sleep(.2)
    except:
        print(f'City not found. Skipping...')
        pass 
print(f'----------------------------------------------')
print(f'Data Retrieval Complete')
print(f'----------------------------------------------')

http://api.openweathermap.org/data/2.5/weather?appid=f563cc6b4ded9445677bab30a69c5435&units=metric&q=London
Beginning Data Retrieval 
Processing record 1 of set 1 | punta arenas
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 2 of set 1 | port blair
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 3 of set 1 | ushuaia
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 4 of set 1 | kamina
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 5 of set 1 | taolanaro
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 6 of set 1 | castro
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 7 of set 1 | matay
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 8 of set 1 | east london
http://api.o

Processing record 21 of set 2 | ostrovnoy
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 22 of set 2 | grindavik
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 23 of set 2 | pyapon
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 24 of set 2 | lesnikovo
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 25 of set 2 | san cristobal
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 26 of set 2 | ampanihy
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 27 of set 2 | bolungarvik
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 28 of set 2 | maniitsoq
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 29 of set 2 | itoman
http://api.open

Processing record 40 of set 3 | georgetown
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 41 of set 3 | ponta do sol
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 42 of set 3 | saint-philippe
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 43 of set 3 | sorong
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 44 of set 3 | vila franca do campo
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 45 of set 3 | sitges
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 46 of set 3 | port hedland
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 47 of set 3 | jiuquan
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 48 of set 3 | saskylakh

Processing record 10 of set 5 | cape town
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 11 of set 5 | kalghatgi
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 12 of set 5 | road town
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 13 of set 5 | partizanskoye
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 14 of set 5 | hornepayne
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 15 of set 5 | severo-kurilsk
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 16 of set 5 | usinsk
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 17 of set 5 | bathsheba
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 18 of set 5 | kulhudhuffushi
ht

Processing record 30 of set 6 | barranco
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 31 of set 6 | alice springs
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 32 of set 6 | ludwigslust
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 33 of set 6 | tres lagoas
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 34 of set 6 | mpanda
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 35 of set 6 | iqaluit
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 36 of set 6 | inta
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 37 of set 6 | amga
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 38 of set 6 | tres de maio
http://api.openweat

Processing record 50 of set 7 | vanimo
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 0 of set 8 | saint george
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 1 of set 8 | opuwo
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 2 of set 8 | eureka
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 3 of set 8 | constitucion
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 4 of set 8 | swellendam
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 5 of set 8 | ladario
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 6 of set 8 | tazovskiy
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 7 of set 8 | benguela
http://api.openweathermap.or

Processing record 22 of set 9 | tessalit
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 23 of set 9 | marsa matruh
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 24 of set 9 | bonoua
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 25 of set 9 | chuy
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 26 of set 9 | guerrero negro
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 27 of set 9 | lorengau
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 28 of set 9 | kendari
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 29 of set 9 | zunyi
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 30 of set 9 | asau
http://api.openweathermap.o

Processing record 42 of set 10 | saint-denis
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 43 of set 10 | mairang
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 44 of set 10 | dwarka
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 45 of set 10 | manicore
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 46 of set 10 | ketchikan
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 47 of set 10 | trofors
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 48 of set 10 | balimo
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 49 of set 10 | yuanping
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 50 of set 10 | weinan
http://api.openwea

Processing record 11 of set 12 | durant
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 12 of set 12 | caravelas
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 13 of set 12 | manica
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 14 of set 12 | sambava
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 15 of set 12 | adrar
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 16 of set 12 | ust-omchug
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 17 of set 12 | ruatoria
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 18 of set 12 | bangassou
http://api.openweathermap.org/data/2.5/weather?
City not found. Skipping...
Processing record 19 of set 12 | samsun
http://api.openweath

## Convert Raw Data to DataFrame
* Save as csv

In [9]:
#weather_dict = {
#WeatherPy_df =pd.DataFrame(columns = ["City","Cloudiness", "Country",
     #                               "Date", "Max Temp", "Wind Speed", 
      #                              "Lat","Lng", "Humidity"])
city_data_df = pd.DataFrame({'City': new_cities,
                            'Latitude': lat,
                            'Longitude': lng,
                            'Temperature': temp,
                            'Humidity': humidity,
                            'Cloudiness': cloudiness,
                            'Wind Speed': wind})
pd.DataFrame.to_csv(city_data_df, 'city_data.csv')

city_data_df.head()
#WeatherPy_df.count()
    #"Cloudiness": cloudiness,
   # "Country": country,
    #"Date": date,
    #"Humidity": humidity,
    #"Lat": lat,
    #"Lng": lng,
    #"Max Temp": temp,
    #"Wind Speed": wind
#}

#weather_data = pd.DataFrame(weather_dict)
#weather_data.to_csv('WeatherPy_data.csv')
 
#print(f'City          {len(new_cities)}')
#print(f'Cloudiness    {len(cloudiness)}')
#print(f'Country    {len(country)}')
#print(f'Date    {len(date)}')
#print(f'Humidity    {len(humidity)}')
#print(f'Lat    {len(lat)}')
#print(f'Lng    {len(lng)}')
#print(f'Max Temp    {len(temp)}')
#print(f'Wind Speed    {len(wind)}')
#weather_data.head()
#newcities_df = pd.DataFrame(new_cities)
#print(newcities_df)
#print (len(weather_data))
#weather_dict = {
   #"City": new_cities,
  # "Cloudiness" : cloudiness,
   #"Country" : country,
   #"Humidity" : humidity,
   #"Lat" : lat,
   #"Lng" : lng,  
   #"Max Temperature" : temp,
   #"Wind Speed" : wind

#weather_data = pd.DataFrame(weather_dict)
#weather_data.count()
#weather_df = pd.DataFrame(weather_dict)
#weather_df.count()
#cities_df=pd.DataFrame(cities)
#cities_df.columns=["City"]
#cities_df['Cloudiness'] = 'cloudiness'
#cities_df['Country'] = 'country'
#cities_df['Date'] = 'date'
#cities_df['Humidity'] = 'humidity'
#cities_df['Lat'] = 'lat'
#cities_df['Lng'] = 'lng'
#cities_df['Max Temp'] = 'temp'
#cities_df['Wind Speed'] = 'wind'
#cities_df.head()
#cities_df.count()

ValueError: arrays must all be same length

In [10]:
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "metric"
# Build partial query URL
query_url = f"{url}appid={Weathermap_API_key}&units={units}&q="
print(query_url+"London")
data=[]
i=0
for city in cities[:50]:
#    print(city)
    json = requests.get(query_url + city).json()
    i=i+1
    try:
        tempmax=json["main"]['temp_max']
        humidity=json["main"]["humidity"]
        speed=json['wind']["speed"]
        cloudiness=json["clouds"]["all"]
        lat=json["coord"]["lat"]
        country=json['sys']["country"]
        date=json['dt']
        lon=json["coord"]["lon"]
        data.append([city,cloudiness,country,date,humidity,lat,lon,tempmax,speed])
#        data.append([city,temp,temp1,temp2,temp3,temp6,temp4,temp5,temp7])
        print(f"counter {i} || {city}")
        time.sleep(.1)
    except:
        print("city not found")

http://api.openweathermap.org/data/2.5/weather?appid=f563cc6b4ded9445677bab30a69c5435&units=metric&q=London
counter 1 || punta arenas
counter 2 || port blair
counter 3 || ushuaia
counter 4 || kamina
city not found
counter 6 || castro
counter 7 || matay
counter 8 || east london
counter 9 || mataura
counter 10 || jamestown
counter 11 || tiarei
counter 12 || cherskiy
counter 13 || mabama
counter 14 || port augusta
counter 15 || puerto ayora
counter 16 || vaini
counter 17 || tolaga bay
counter 18 || rawson
counter 19 || tiksi
counter 20 || iskateley
city not found
city not found
counter 23 || danilovka
counter 24 || tupiza
counter 25 || turayf
counter 26 || hobart
counter 27 || valparaiso
city not found
counter 29 || kapaa
city not found
counter 31 || bengkulu
counter 32 || katsuura
city not found
counter 34 || labuhan deli
counter 35 || maceio
counter 36 || bredasdorp
counter 37 || ribeira grande
counter 38 || bonthe
counter 39 || pangnirtung
counter 40 || chokurdakh
counter 41 || iluliss

In [11]:
data_df=pd.DataFrame(data,columns=["City","Cloudiness","Country", "Date","Humidity","Lat","Lng","Max Temp","Wind Speed"])
data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,punta arenas,75,CL,1603722564,50,-53.15,-70.92,13.00,8.20
1,port blair,76,IN,1603722759,81,11.67,92.75,26.87,2.18
2,ushuaia,75,AR,1603722514,66,-54.80,-68.30,10.00,2.10
3,kamina,89,CD,1603722760,60,-8.74,24.99,27.80,1.37
4,castro,81,BR,1603722513,54,-24.79,-50.01,24.33,2.38


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

### Latitude vs. Temperature Plot

In [ ]:

weather_df = pd.DataFrame(weather_dict)
plt.scatter(weather_df["Lat"], weather_df["Temp"], marker = "o", s=10)
plt.title(f"City Latitude vs. Max Temperature")
plt.ylabel("Temperature (F)")
plt.axvline(0, color = 'black', alpha = .25, label = 'Equator')
plt.text(1,30, 'Equator', rotation=90)
plt.ylim(-50,120)
plt.xlabel("Latitude")
plt.xlim(-60,90)
plt.grid(True)
plt.show()



### Latitude vs. Humidity Plot

In [ ]:
plt.scatter(weather_df['Lat'], weather_df['Humidity'], marker="o", facecolors="blue", edgecolors="black")
plt.title("City Latitude vs. Humidity")
plt.xlabel("Latitude")
plt.ylabel("Humidity(%)")
plt.savefig("Humidity.png")
plt.show()

### Latitude vs. Cloudiness Plot

In [ ]:
plt.scatter(weather_df['Lat'], weather_df['Cloudiness'], marker="o", facecolors="blue", edgecolors="black")
plt.title("City Latitude vs. Cloudiness")
plt.xlabel("Latitude")
plt.ylabel("Cloudiness(%)")
plt.savefig("Cloudiness.png")
plt.show()

### Latitude vs. Wind Speed Plot

In [ ]:
plt.scatter(weather_df['Lat'], weather_df['Wind Speed'], marker="o", facecolors="blue", edgecolors="black")
plt.title("City Latitude vs. Wind Speed")
plt.xlabel("Latitude")
plt.ylabel("Wind Speed(%)")
plt.savefig("Wind Speed.png")
plt.show()

## Linear Regression

### Write linear regression plot function (optional)

In [ ]:
import matplotlib.pyplot as plt
from mlxtend.plotting import plot_linear_regression
import numpy as np
intercept, slope, corr_coeff = plot_linear_regression(X, y)
plt.show()

###  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
xlbl = "Lat"
ylbl = "Temp"
lblpos = (0,25)
plotLinearRegression(northern[xlbl], northern[ylbl], xlbl, ylbl, lblpos, 6)

###  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

In [ ]:
xlbl = "Lat"
ylbl = "Temp"
lblpos = (-55,90)
plotLinearRegression(southern[xlbl], southern[ylbl], xlbl, ylbl, lblpos, 6)

###  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
xlbl = "Lat"
ylbl = "Humidity"
lblpos = (45,10)
plotLinearRegression(northern[xlbl], northern[ylbl], xlbl, ylbl, lblpos, 7)

###  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

In [ ]:
x_values = southern_hemisphere['Lat']
y_values = southern_hemisphere['Humidity']
(slope, intercept, rvalue, pvalue, stderr) = linregress(x_values, y_values)
regress_values = x_values * slope + intercept
line_eq = "y =" + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values, y_values)
plt.plot(x_values, regress_values, "r-")
plt.annotate(line_eq,(-25,10), fontsize=15, color ="red")
plt.ylim (0,100)
plt.xlim (-60,0)
plt.ylabel("Humidity")
plt.xlabel("Latitude")


###  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
crit_north = weather_df.Lat >= 0
crit_south = weather_df.Lat < 0
north_weather = weather_df[crit_north]
south_weather = weather_df[crit_south]
north_weather = north_weather.reset_index(drop=True)
south_weather = south_weather.reset_index(drop=True)
make_lin_reg_plot(north_weather["Lat"], north_weather["Cloudiness"],\
                                                                      'Lat', f'Cloudiness (%)', 'Northern Hemisphere', 40,10)
print(weather_df)                 

###  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

In [ ]:
crit_north = weather_df.Lat >= 0
crit_south = weather_df.Lat < 0
north_weather = weather_df[crit_north]
south_weather = weather_df[crit_south]
north_weather = north_weather.reset_index(drop=True)
south_weather = south_weather.reset_index(drop=True)
make_lin_reg_plot(south_weather["Lat"], south_weather["Cloudiness"],\
                                                                      'Lat', f'Cloudiness (%)', 'Northern Hemisphere', -60,10)
print(weather_df)                 

###  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
make_lin_reg_plot(north_weather["lat"], north_weather["wind"],\
                                                              'Lat', f'Wind Speed (mph)', 'Northern Hemisphere',\ 
                  10,30)

###  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [ ]:
make_lin_reg_plot(south_weather["lat"], south_weather["wind"],\
                                                              'Lat', f'Wind Speed (mph)', 'Southern Hemisphere',\ 
                  10,30)